In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.stats import poisson
import statistics as st
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

print(tf.__version__)

2.6.0


In [3]:
upcoming_caf = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CONCACAF/INPUT/upcoming_concacaf.csv")
caf = pd.read_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CONCACAF/INPUT/concacaf_input.csv")
# home_xg = conmebol["H_xG"]
# away_xg = conmebol["A_xG"]

In [4]:
# Regression model - Neural network for home_xG, away_xg
home_xg_df = caf[['home_score', 'H_attack', 'A_defence']]
away_xg_df = caf[['away_score', 'H_defence', 'A_attack']]

# Splitting dataset
train_homexG = home_xg_df.sample(frac=0.8, random_state = 0)
test_homexG = home_xg_df.drop(train_homexG.index)

train_awayxG = away_xg_df.sample(frac=0.8, random_state = 0)
test_awayxG = away_xg_df.drop(train_awayxG.index)

# For home_xG
train_features = train_homexG.copy()
test_features = test_homexG.copy()

train_labels = train_features.pop('home_score')
test_labels = test_features.pop('home_score')

# For away_xG
train_features2 = train_awayxG.copy()
test_features2 = test_awayxG.copy()

train_labels2 = train_features2.pop('away_score')
test_labels2 = test_features2.pop('away_score')

In [5]:
# Pre-processing, home / away
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

normalizer2 = preprocessing.Normalization(axis=-1)
normalizer2.adapt(np.array(train_features2))

# Model functions, home / away
# home
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])
# away
linear_model2 = tf.keras.Sequential([
    normalizer2,
    layers.Dense(units=1)
])

def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.003))
  return model

# Building model, home / away
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

dnn_model2 = build_and_compile_model(normalizer2)
dnn_model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 2)                 5         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                192       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 4,422
Trainable params: 4,417
Non-trainable params: 5
_________________________________________________________________
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_1 (Normalizati (None, 2)                 5         
______________________________

In [6]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1500)

# training model home

CPU times: user 33.1 s, sys: 3.96 s, total: 37.1 s
Wall time: 31 s


In [7]:
%%time
history = dnn_model2.fit(
    train_features2, train_labels2,
    validation_split=0.2,
    verbose=0, epochs=1500)

# training model away

CPU times: user 31.7 s, sys: 4.07 s, total: 35.8 s
Wall time: 27.6 s


In [8]:
test_results = {}
test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)
pd.DataFrame(test_results, index=['Mean absolute error [home_score]']).T

test_results2 = {}
test_results2['dnn_model2'] = dnn_model2.evaluate(test_features2, test_labels2, verbose=0)
pd.DataFrame(test_results2, index=['Mean absolute error [away_score]']).T


,Mean absolute error [away_score]
dnn_model2,0.790059


In [9]:
# Predicting home_xG and away_xG for upcoming fixtures
home_xg_features = upcoming_caf[['H_attack', 'A_defence']]
pred_home_xg = dnn_model.predict(home_xg_features).flatten()
pred_home_xg = [0 if i < 0 else i for i in pred_home_xg]

away_xg_features = upcoming_caf[['H_defence', 'A_attack']]
pred_away_xg = dnn_model2.predict(away_xg_features).flatten()
pred_away_xg = [0 if i < 0 else i for i in pred_away_xg]
#pred_away_xg

In [10]:
# Run the poisson results for the home team
home_outcome = []

for i in range(len(pred_home_xg)):
    expected = pred_home_xg[i] + 0.5
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    home_outcome.append(common)

home_outcome

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [11]:
# Run the poisson results for the away team
away_outcome = []

for i in range(len(pred_away_xg)):
    expected = pred_away_xg[i]
    poi = poisson.rvs(expected, size=10000)
    # Select most common outcome and append to the poisson outcome list
    common = st.mode(poi)
    away_outcome.append(common)

away_outcome

[1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 0]

In [12]:
upcoming_caf["Poisson_H_xG"] = home_outcome
upcoming_caf["Poisson_A_xG"] = away_outcome
caf_result = upcoming_caf[['Date', 'Home', 'Away', 'Poisson_H_xG', 'Poisson_A_xG']]

result = []
outcomes = ["H_WIN", "DRAW", "A_WIN"]
for i in range(len(away_outcome)):
    if upcoming_caf["Poisson_H_xG"][i] > upcoming_caf["Poisson_A_xG"][i]:
        result.append(outcomes[0])
    elif upcoming_caf["Poisson_H_xG"][i] == upcoming_caf["Poisson_A_xG"][i]:
        result.append(outcomes[1])
    else:
        result.append(outcomes[2])

caf_result["Results"] = result

H_Points = []
A_Points = []
for i in range(len(away_outcome)):
    if result[i] == "H_WIN":
        H_Points.append(3)
        A_Points.append(0)
    elif result[i] == "DRAW":
        H_Points.append(1)
        A_Points.append(1)
    else:
        H_Points.append(0)
        A_Points.append(3)
       
        
caf_result["H_Points"] = H_Points
caf_result["A_Points"] = A_Points
caf_result

,Date,Home,Away,Poisson_H_xG,Poisson_A_xG,Results,H_Points,A_Points
0,07/10/2021,United States,Jamaica,0,1,A_WIN,0,3
1,07/10/2021,Honduras,Costa Rica,0,0,DRAW,1,1
2,07/10/2021,Mexico,Canada,0,0,DRAW,1,1
3,07/10/2021,El Salvador,Panama,0,1,A_WIN,0,3
4,10/10/2021,Costa Rica,El Salvador,0,1,A_WIN,0,3
5,10/10/2021,Jamaica,Canada,0,0,DRAW,1,1
6,10/10/2021,Panama,United States,0,1,A_WIN,0,3
7,10/10/2021,Mexico,Honduras,0,0,DRAW,1,1
8,13/10/2021,United States,Costa Rica,0,0,DRAW,1,1
9,13/10/2021,Canada,Panama,0,1,A_WIN,0,3


In [13]:
# Create Points Table
result_table = pd.DataFrame(columns=['Team','GF','GA', 'Points'])
caf_home = caf_result.groupby("Home")["H_Points"].sum()
caf_away = caf_result.groupby("Away")["A_Points"].sum()
points = caf_home + caf_away
H_GF = caf_result.groupby("Home")["Poisson_H_xG"].sum()
A_GF = caf_result.groupby("Away")["Poisson_A_xG"].sum()
H_GA = caf_result.groupby("Home")["Poisson_A_xG"].sum()
A_GA = caf_result.groupby("Away")["Poisson_H_xG"].sum()

GF = H_GF + A_GF
GA = H_GA + A_GA

team_list = pd.unique(caf_result["Home"])
team_list_sort = sorted(team_list)

result_table["Points"] = points
result_table["GF"] = GF
result_table["GA"] = GA
result_table["Team"] = team_list_sort
result_table

,Team,GF,GA,Points
Home,,,,
Canada,Canada,0,3,9
Costa Rica,Costa Rica,5,2,15
El Salvador,El Salvador,8,3,19
Honduras,Honduras,0,2,9
Jamaica,Jamaica,2,4,12
Mexico,Mexico,1,2,12
Panama,Panama,2,1,14
United States,United States,3,4,14


In [14]:
#caf_result.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/OUTPUT/caf_results.csv", index=False)
#result_table.to_csv("/Users/abhijotsingh/Desktop/AtlassianDatathon/CAF/OUTPUT/caf_points_table.csv", index=False)